## **I) IMPORTS & EXPLORATION**

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

### **1) DATASETS URLS**

In [2]:
namebasics_url = "https://datasets.imdbws.com/name.basics.tsv.gz"
akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"
basics_url = "https://datasets.imdbws.com/title.basics.tsv.gz"
crew_url = "https://datasets.imdbws.com/title.crew.tsv.gz"
principals_url = "https://datasets.imdbws.com/title.principals.tsv.gz"
ratings_url ="https://datasets.imdbws.com/title.ratings.tsv.gz"

### **2) LOADING DATASETS & CLEANING**

#### **2.1 ratings dataset**

In [ ]:
# ratings dataset loading: 
ratings = pd.read_csv(ratings_url, sep='\t', na_values='\\N', dtype={'averageRating': float, 'numVotes': int, 'tconst': str})
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2187
1,tt0000002,5.5,307
2,tt0000003,6.5,2275
3,tt0000004,5.1,196
4,tt0000005,6.2,3012


In [ ]:
# Exploration of ratings dataset
print(ratings.info())
print(ratings.duplicated().sum())
print(ratings.isnull().sum())
ratings['numVotes'].describe().round(2)
ratings['averageRating'].describe().round(2)

#### **2.2 basics dataset**

In [ ]:
# basics dataset loading: 
basics = pd.read_csv(
	basics_url,
	sep='\t',
	na_values='\\N',
	dtype={
		'tconst': str,
		'titleType': str,
		'primaryTitle': str,
		'originalTitle': str,
		'isAdult': 'Int64',
		'startYear': 'Int64',
		'endYear': str,
		'runtimeMinutes': str,
		'genres': str
	}
)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892,NaN,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,Short


In [ ]:
# filter isAdult movies
# filter startYear starting from 1980 to today(year()) (adapting to users' age)
# filter movies include only movies (not documentaries, shorts, tvshows or series)
current_year = pd.Timestamp.now().year
basics = basics[(basics['isAdult'] == 0) &
                (basics['startYear'] >= 1980) & 
                (basics['startYear'] <= current_year) &
                (basics['titleType'] == 'movie')
                ]

In [ ]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 450436 entries, 11630 to 12169530
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          450436 non-null  object
 1   titleType       450436 non-null  object
 2   primaryTitle    450434 non-null  object
 3   originalTitle   450434 non-null  object
 4   isAdult         450436 non-null  Int64 
 5   startYear       450436 non-null  Int64 
 6   endYear         0 non-null       object
 7   runtimeMinutes  346279 non-null  object
 8   genres          417034 non-null  object
dtypes: Int64(2), object(7)
memory usage: 35.2+ MB


#### **2.3 akas dataset**

In [ ]:
# akas dataset loading:
akas = pd.read_csv(akas_url, 
                   sep='\t', 
                   na_values='\\N', 
                   dtype={'titleId': str, 
                          'ordering': int, 
                          'title': str, 
                          'region': str, 
                          'language': str, 
                          'types': str, 
                          'attributes': str, 
                          'isOriginalTitle': 'Int64'})
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Carmencita,NaN,NaN,original,NaN,1
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita,US,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0


In [42]:
# filter region 'FR' for France
akas = akas[akas['region'] == 'FR']
akas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5306431 entries, 12 to 54384396
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int32 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  Int64 
dtypes: Int64(1), int32(1), object(6)
memory usage: 349.2+ MB


#### **2.4 principals dataset**

In [43]:
principals = pd.read_csv(principals_url, 
                         sep='\t',
                         na_values='\\N',
                         dtype={'tconst': str,
                                'ordering': int,
                                'nconst': str,
                                'category': str,
                                'job': str,
                                'characters': str})
principals.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,NaN,"[""Self""]"
1,tt0000001,2,nm0005690,director,NaN,NaN
2,tt0000001,3,nm0005690,producer,producer,NaN
3,tt0000001,4,nm0374658,cinematographer,director of photography,NaN
4,tt0000002,1,nm0721526,director,NaN,NaN


In [52]:
# filter principals to keep only actors and actresses and directors
# keep only theses columns : tconst, nconst, category
# remove duplicates if any
# check for nulls (no nulls) 
principals = principals[principals['category'].isin(['actor', 'actress', 'director'])]
principals = principals[['tconst', 'nconst', 'category']]
principals = principals.drop_duplicates()
principals.info()


<class 'pandas.core.frame.DataFrame'>
Index: 46934247 entries, 1 to 96798092
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   tconst    object
 1   nconst    object
 2   category  object
dtypes: object(3)
memory usage: 1.4+ GB


## **II) FINAL DATASETS** 

In [ ]:
# We keep and export the cleaned datasets: ratings and principals (distinguish actors/actresses and directors)
# Merge basics and akas into a new dataframe called movies
# Create 
movies = pd.merge(basics, akas, left_on='tconst', right_on='tconst', how='inner')